In [2]:
//외부 라이브러리 필요
//자연어 처리 예시 LSA
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.spark.ml.feature.{CountVectorizer , IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import scala.collection.JavaConverters._
import scala.collection.mutable.ArrayBuffer
import java.util.Properties
import java.io.ByteArrayInputStream;

import edu.umd.cloud9.collection.XMLInputFormat
import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}

import info.bliki.wiki.dump._
import org.xml.sax.SAXException
import spark.implicits._

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.spark.ml.feature.{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import scala.collection.JavaConverters._
import scala.collection.mutable.ArrayBuffer
import java.util.Properties
import java.io.ByteArrayInputStream
import edu.umd.cloud9.collection.XMLInputFormat
import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}
import info.bliki.wiki.dump._
import org.xml.sax.SAXException
import spark.implicits._


In [3]:
val path = "Data/Wikipedia-20211016122215.xml"
@transient val conf = new Configuration()
conf.set(XMLInputFormat.START_TAG_KEY, "<page>")
conf.set(XMLInputFormat.END_TAG_KEY, "</page>")
val kvs = spark.sparkContext.newAPIHadoopFile(path, classOf[XMLInputFormat], classOf[LongWritable], classOf[Text], conf)
val rawXmls = kvs.map(_._2.toString).toDS()

//경로 지정 후 위키피디아 XML 파일 덤프

path: String = Data/Wikipedia-20211016122215.xml
conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
kvs: org.apache.spark.rdd.RDD[(org.apache.hadoop.io.LongWritable, org.apache.hadoop.io.Text)] = Data/Wikipedia-20211016122215.xml NewHadoopRDD[0] at newAPIHadoopFile at <console>:51
rawXmls: org.apache.spark.sql.Dataset[String] = [value: string]


In [9]:
//문서의 형식은 날리고 퓨어 텍스트만 남기는 과정

case class Page(var page: WikiArticle = new WikiArticle) {}

class ArticleFilter(val Page: Page) extends IArticleFilter{
    @throws(classOf[SAXException])
    def process(page: WikiArticle, siteinfo: Siteinfo){
        Page.page = page
    }
}

def wikiXmlToPlainText(pageXml: String): Option[(String, String)] = {
    val Page = new Page
    try{
        val parser = new WikiXMLParser(new ByteArrayInputStream(pageXml.getBytes), new ArticleFilter(Page))
        parser.parse()
    } catch{
        case e: Exception =>
    }
    val page = Page.page
    if (page.getText != null&&page.getTitle != null
       && page.getId != null && page.getRevisionId != null
       && page.getTimeStamp != null && !page.isTemplate) {
        Some((page.getTitle, page.getText))
    } else {
        None
    }
}
val doctTexts = rawXmls.filter(_ != null).flatMap(wikiXmlToPlainText)

<console>: 42: error: Unable to find encoder for type (String, String). An implicit Encoder[(String, String)] is needed to store (String, String) instances in a Dataset. Primitive types (Int, String, etc) and Product types (case classes) are supported by importing spark.implicits._  Support for serializing other types will be added in future releases.